##Aumento de imagenes y generación de archivos json

#####Input: imagenes y archivos .xml o .json desde Github
#####Output: imagenes aumentadas y json de las mismas a Github

####Params

In [ ]:
generate_images_count =  3#@param {type:"integer"}

# mostrar imagenes originales y sus transformaciones, para debug
show_generated_images = True #@param {type:"boolean"}
images_to_show =  15#@param {type:"integer"}

images_repo_url = 'https://github.com/cololaborde/etiquetado' #@param {type:"string"}

import os
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(images_repo_url)))
path_annotations = repo_dir_path+'/annotations' 
path_images_train = repo_dir_path+'/Guille-Tomas/original/train'
path_images_test = repo_dir_path+'/Guille-Tomas/original/test' 

github_pass = 'colo41212209' #@param {type:"string"}
github_mail = 'cololaborde@gmail.com' #@param {type:"string"}
github_user = 'cololaborde' #@param {type:"string"}
repo_name = 'etiquetado' #@param {type:"string"}
labeled_with = 'labelme (poligonos)' #@param ["labelimg", "labelme (poligonos)"]

repo = 'https://'+github_user + ':' +github_pass+'@github.com/'+github_user+'/'+repo_name+'.git'

####Clonamos repo con imagenes

In [ ]:
%cd /content
!git clone {repo}

/content
Cloning into 'etiquetado'...
remote: Enumerating objects: 830, done.
remote: Counting objects: 100% (830/830), done.
remote: Compressing objects: 100% (493/493), done.
remote: Total 830 (delta 409), reused 758 (delta 337), pack-reused 0
Receiving objects: 100% (830/830), 28.03 MiB | 27.10 MiB/s, done.
Resolving deltas: 100% (409/409), done.


In [ ]:
print(repo_dir_path)
print(path_annotations)
print(path_images_test)
print(path_images_train)

/content/etiquetado
/content/etiquetado/annotations
/content/etiquetado/Guille-Tomas/original/test
/content/etiquetado/Guille-Tomas/original/train


####Instalamos libs

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%tensorflow_version 1.x
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 
import imageio
import pandas as pd
import numpy as np
import re
import os
import glob
import xml.etree.ElementTree as ET
import shutil

import json
import csv

/content
TensorFlow 1.x selected.
/content/models/research


####Funciones

In [ ]:
# Function that will extract column data for our CSV file as pandas DataFrame
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df
   
# apply the function to convert all XML files in images/ folder into labels.csv
if os.path.exists(path_annotations):
  shutil.rmtree(path_annotations)

os.mkdir(path_annotations) 

In [ ]:
# Function that will extract column data for our CSV file as pandas DataFrame
def json_to_csv(path):
    json_list = []
    for json_file in glob.glob(path + '/*.json'):
        json_read = open(json_file)
        json_loaded = json.load(json_read)
        #save width and height
        height = json_loaded["imageHeight"]
        width = json_loaded["imageWidth"]
        filename = json_loaded["imagePath"]
        #iterate labels and points
        for elements in json_loaded["shapes"]:
            label = elements["label"]
            points = elements["points"]
            xmin = round(points[0][0]) + 1
            xmax = round(points[2][0]) + 1
            ymin = round(points[1][1]) + 1
            try:
              ymax = round(points[3][1]) + 1
            except:
              print('Este archivo requiere intervención manual, debido a que contiene poligonos con menos de 4 puntos: ' + str(json_file))
            value = (filename, width, height, label, xmin, ymin, xmax, ymax)
            json_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    json_df = pd.DataFrame(json_list, columns=column_name)
    return json_df
   
# apply the function to convert all XML files in images/ folder into labels.csv
if os.path.exists(path_annotations):
  shutil.rmtree(path_annotations)

os.mkdir(path_annotations) 

In [ ]:
# function to convert BoundingBoxesOnImage object into DataFrame
def bbs_obj_to_df(bbs_object):
#     convert BoundingBoxesOnImage object into array
    bbs_array = bbs_object.to_xyxy_array()
#     convert array into a DataFrame ['xmin', 'ymin', 'xmax', 'ymax'] columns
    df_bbs = pd.DataFrame(bbs_array, columns=['xmin', 'ymin', 'xmax', 'ymax'])
    return df_bbs

In [ ]:
def image_aug(df, images_path, aug_images_path, image_prefix, augmentor):
    # create data frame which we're going to populate with augmented image info
    aug_bbs_xy = pd.DataFrame(columns=
                              ['filename','width','height','class', 'xmin', 'ymin', 'xmax', 'ymax']
                             )
    grouped = df.groupby('filename')
    
    for filename in df['filename'].unique():
    #   get separate data frame grouped by file name
        group_df = grouped.get_group(filename)
        group_df = group_df.reset_index()
        group_df = group_df.drop(['index'], axis=1)   
    #   read the image
        image = imageio.imread(images_path+filename)
    #   get bounding boxes coordinates and write into array        
        bb_array = group_df.drop(['filename', 'width', 'height', 'class'], axis=1).values
    #   pass the array of bounding boxes coordinates to the imgaug library
        bbs = BoundingBoxesOnImage.from_xyxy_array(bb_array, shape=image.shape)
    #   apply augmentation on image and on the bounding boxes
        image_aug, bbs_aug = augmentor(image=image, bounding_boxes=bbs)
    #   disregard bounding boxes which have fallen out of image pane    
        bbs_aug = bbs_aug.remove_out_of_image()
    #   clip bounding boxes which are partially outside of image pane
        bbs_aug = bbs_aug.clip_out_of_image()
        
    #   don't perform any actions with the image if there are no bounding boxes left in it    
        if re.findall('Image...', str(bbs_aug)) == ['Image([]']:
            pass
        
    #   otherwise continue
        else:
        #   write augmented image to a file
            imageio.imwrite(aug_images_path+image_prefix+filename, image_aug)  
        #   create a data frame with augmented values of image width and height
            info_df = group_df.drop(['xmin', 'ymin', 'xmax', 'ymax'], axis=1)    
            for index, _ in info_df.iterrows():
                info_df.at[index, 'width'] = image_aug.shape[1]
                info_df.at[index, 'height'] = image_aug.shape[0]
        #   rename filenames by adding the predifined prefix
            info_df['filename'] = info_df['filename'].apply(lambda x: image_prefix+x)
        #   create a data frame with augmented bounding boxes coordinates using the function we created earlier
            bbs_df = bbs_obj_to_df(bbs_aug)
        #   concat all new augmented info into new data frame
            aug_df = pd.concat([info_df, bbs_df], axis=1)
        #   append rows to aug_bbs_xy data frame
            aug_bbs_xy = pd.concat([aug_bbs_xy, aug_df])            
    
    # return dataframe with updated images and bounding boxes annotations 
    aug_bbs_xy = aug_bbs_xy.reset_index()
    aug_bbs_xy = aug_bbs_xy.drop(['index'], axis=1)
    return aug_bbs_xy

In [ ]:
if(labeled_with == 'labelme (poligonos)'):
  train_labels_df = json_to_csv(path_images_train)
  train_labels_df.to_csv((path_annotations+'/temp_train_labels.csv'), index=None)

  test_labels_df = json_to_csv(path_images_test)
  test_labels_df.to_csv((path_annotations+'/temp_test_labels.csv'), index=None)
else:
  train_labels_df = xml_to_csv(path_images_train)
  train_labels_df.to_csv((path_annotations+'/temp_train_labels.csv'), index=None)

  test_labels_df = xml_to_csv(path_images_test)
  test_labels_df.to_csv((path_annotations+'/temp_test_labels.csv'), index=None)

Este archivo requiere intervención manual, debido a que contiene poligonos con menos de 4 puntos: /content/etiquetado/Guille-Tomas/original/train/EDF 07-EST-06.json
Este archivo requiere intervención manual, debido a que contiene poligonos con menos de 4 puntos: /content/etiquetado/Guille-Tomas/original/train/EDF 07-EST-06.json


las transformaciones son las de la libreria albumnations, que es un wrapper de imgaug .. las posibles opciones son las mencionadas en https://albumentations.readthedocs.io/en/latest/api/index.html

In [ ]:
aug = iaa.SomeOf(2, [    
    iaa.Affine(scale=(0.5, 1.5)),
    #iaa.Affine(rotate=(-60, 60)),
    iaa.Affine(translate_percent={"x":(-0.3, 0.3),"y":(-0.3, 0.3)}),
    iaa.Fliplr(1),
    iaa.GaussianBlur(sigma=(1.0, 3.0)),
    iaa.AdditiveGaussianNoise(scale=(0.03*255, 0.05*255))
])

In [ ]:
if os.path.exists(path_images_train+'/aug_images'):
  shutil.rmtree(path_images_train+'/aug_images')
os.mkdir(path_images_train+'/aug_images') 

if os.path.exists(path_images_test+'/aug_images'):
  shutil.rmtree(path_images_test+'/aug_images')
os.mkdir(path_images_test+'/aug_images') 

####Aumentamos imagenes

In [ ]:
# initialize empty DataFrame
augmented_images_train_df = pd.DataFrame(columns=['filename','width','height','class','xmin','ymin','xmax','ymax'])
# apply augmentation function 5 times to the same set of images
for i in range(generate_images_count):
    aug_df = image_aug(train_labels_df, path_images_train+'/', path_images_train+'/aug_images/', 'aug'+str(i)+'_', aug)
    augmented_images_train_df = pd.concat([augmented_images_train_df, aug_df])

###Mostramos imagenes

In [ ]:
if show_generated_images:
  %matplotlib inline
  import matplotlib as mpl
  import matplotlib.pyplot as plt
  import PIL.ImageDraw as ImageDraw
  IMAGE_SIZE = (16, 16)

  i = 0

  for filename in train_labels_df['filename'].unique():
    
    if i == images_to_show:
      break;

    asd = augmented_images_train_df[augmented_images_train_df["filename"].str.contains(filename)]
    
    for filename1 in asd['filename'].unique():
      grouped = asd.groupby('filename')
      group_df = grouped.get_group(filename1)
    
      plt.figure(figsize=IMAGE_SIZE)
      image = imageio.imread(path_images_train+'/aug_images/'+filename1)
      plt.imshow(image)

      for index, row in group_df.iterrows():
          coord = [[row['xmin'], row['ymax']], [row['xmax'], row['ymax']], [row['xmax'], row['ymin']], [row['xmin'], row['ymin']]]
         
          coord.append(coord[0]) #repeat the first point to create a 'closed loop'
          coord.append(coord[1])
          coord.append(coord[2])
          coord.append(coord[3])
          xs, ys = zip(*coord) #create lists of x and y values
          plt.plot(xs,ys, linewidth=2) 
    
    i+=1

###Aumento

In [ ]:
# initialize empty DataFrame
augmented_images_test_df = pd.DataFrame(columns=['filename','width','height','class','xmin','ymin','xmax','ymax'])
# apply augmentation function 5 times to the same set of images
for i in range(int(generate_images_count)):
    aug_df = image_aug(test_labels_df, path_images_test+'/', path_images_test+'/aug_images/', 'aug'+str(i)+'_', aug)
    augmented_images_test_df = pd.concat([augmented_images_test_df, aug_df])

In [ ]:
all_labels_train_df = pd.concat([train_labels_df, augmented_images_train_df])
all_labels_train_df.to_csv(path_annotations+'/train_labels.csv', index=False)


all_labels_test_df = pd.concat([test_labels_df, augmented_images_test_df])
all_labels_test_df.to_csv(path_annotations+'/test_labels.csv', index=False)

In [ ]:
''' for file in os.listdir(path_images_train+'/aug_images/'):
    shutil.copy(path_images_train+'/aug_images/'+file, path_images_train+'/'+file)

for file in os.listdir(path_images_test+'/aug_images/'):
    shutil.copy(path_images_test+'/aug_images/'+file, path_images_test+'/'+file) '''

####Eliminamos recursos innecesarios

In [ ]:
# clear resources
!rm '{path_annotations}/temp_train_labels.csv'
!rm '{path_annotations}/temp_test_labels.csv'

###Generamos los .json para las imagenes aumentadas (labelme) o para todas las imagenes (labelimg)

In [ ]:
def csv_to_json(csv_to_open, path_to_save) :

    files = {}
    all_data = {}   
    data = []

    with open(csv_to_open, mode='r') as csv_file: #me quedo con los nombres de las imagenes, sin repetidos
      csv_reader = csv.DictReader(csv_file)
      for row in csv_reader:
        if(labeled_with == 'labelme'): #si es etiquetado con labelme
          if('aug' in row['filename']): #genero un json por cada imagen aumentada nueva
            files[row['filename']] = ''
        else:                           #si fue etiquetado con labelimg,
          files[row['filename']] = ''   #genero un json para cada imagen

    for i in files:   #por cada correspondiente imagen
      csv_file = open(csv_to_open, mode='r')  
      csv_reader = csv.DictReader(csv_file)
      for row in csv_reader:    #leo el csv con toda la informacion de todas las imagenes
          if(str(i) == str(row['filename'])): #si corresponde
              #guardo la informacion para generar el json
              data.append([row['class'], row['xmin'], row['ymin'], row['xmax'], row['ymax'], row['width'], row['height']]) 
    
      all_data[i] = data
      data = []
    
    write_json = {}


    for image in all_data:
        
        store_json = open(path_to_save + '/' + image[:-4] + '.json', 'w+')

        write_json["verion"] = "3.16.7"
        write_json["flags"] = {}
        write_json["shapes"] = []


        for elem in all_data[image]: # por cada elemento detectado, obtengo xmin ymin xmax ymax
            if (elem[1] != '' and elem[2] != '' and elem[3] != '' and elem[4] != ''):
              p1 = (float(elem[1]))
              p2 = (float(elem[2]))
              p3 = (float(elem[3]))
              p4 = (float(elem[4]))
              write_json["shapes"].append(({"label": elem[0], "line_color": None, "fill_color": None, "points": [  [ p1, p2 ], [ p3, p2 ], [ p3, p4 ], [ p1, p4 ] ] , "shape_type":"polygon", "flags":{} }))

        write_json["line_color"] = [0, 255, 0, 128]
        write_json["fillColor"] = [255, 0, 0, 128]
        write_json["imagePath"] = image
        write_json["imageData"] = ""
        write_json["imageHeight"] = 1280
        write_json["imageWidth"] = 1600

        #escribo el json
        store_json.seek(0)
        json.dump(write_json,store_json,indent=4)
        store_json.truncate()

    print('Conversiones finalizadas...')


In [ ]:
csv_to_open = '/content/' + repo_name + '/annotations/test_labels.csv'
path_to_save = path_images_test +'/aug_images/'
csv_to_json(csv_to_open, path_to_save)
csv_to_open = '/content/'+ repo_name + '/annotations/train_labels.csv'
path_to_save = path_images_train +'/aug_images/'
csv_to_json(csv_to_open, path_to_save)

Conversiones finalizadas...
Conversiones finalizadas...


###Subimos los archivos a github

In [ ]:
%cd {repo_dir_path}
!git remote rm origin
!git init
!git remote add origin {repo}
!git config --global user.email {github_mail}
!git config --global user.name {github_user}

!git add .
''' 
%cd {path_annotations}
#!git add test.record
#!git add train.record
!git add label_map.pbtxt
!git commit -m 'tfrecords actualizados'

%cd {path_images_train+'/aug_images'}
!git add .
%cd {path_images_test+'/aug_images'}
!git add . '''
!git commit -m "aumentadas"


!git push -u origin master  

/content/etiquetado
Reinitialized existing Git repository in /content/etiquetado/.git/
[master 8f678ae] aumentadas
 163 files changed, 90101 insertions(+), 2977 deletions(-)
 create mode 100644 Guille-Tomas/original/test/aug_images/EDF 03 - EST-06.json
 create mode 100644 Guille-Tomas/original/test/aug_images/EDF 07-EST-01.json
 create mode 100644 Guille-Tomas/original/test/aug_images/EDF 07-EST-03.json
 create mode 100644 Guille-Tomas/original/test/aug_images/EDF 07-EST-04.json
 create mode 100644 Guille-Tomas/original/test/aug_images/EDF 07-EST-05.json
 create mode 100644 Guille-Tomas/original/test/aug_images/EDF 08 - EST-02.json
 create mode 100644 Guille-Tomas/original/test/aug_images/aug0_EDF 03 - EST-06.jpg
 create mode 100644 Guille-Tomas/original/test/aug_images/aug0_EDF 03 - EST-06.json
 create mode 100644 Guille-Tomas/original/test/aug_images/aug0_EDF 07-EST-01.jpg
 create mode 100644 Guille-Tomas/original/test/aug_images/aug0_EDF 07-EST-01.json
 create mode 100644 Guille-Tom

###Listo.